In [73]:
from Camera import Camera
import numpy as np
from Frame import Frame
from plyfile import PlyData
import numpy as np
import argparse
from io import BytesIO
import matplotlib.pyplot as plt
import Plotters
import plotly.graph_objects as go
import scipy.io
import pandas as pd
import Utils

%matplotlib qt

# load hull
path = 'G:/My Drive/Research/gs_data/mov19_2022_03_03/'
real_coord = scipy.io.loadmat(f'{path}/3d_pts/real_coord.mat')['all_coords']
points_3d = {body_wing : pd.DataFrame(Utils.load_hull(body_wing,path),columns = ['X','Y','Z','frame']) for body_wing in ['body','rwing','lwing']}
# initilize objects
frames = [1407]#list(range(500,700,13))

image_name,points_in_idx = Utils.define_frames(frames,points_3d)
cameras = {f'cam{cam + 1}':Camera(path,cam) for cam in range(4)}
frames = {f'{im_name}.jpg':Frame(path,im_name,points_in_idx[im_name.split('CAM')[0]],real_coord,idx) for idx,im_name in enumerate(image_name)}

In [74]:


def process_ply_to_splat(ply_file_path):
    pos = []
    color = []
    rot = []
    scales = []

    plydata = PlyData.read(ply_file_path)
    vert = plydata["vertex"]
    sorted_indices = np.argsort(
        -np.exp(vert["scale_0"] + vert["scale_1"] + vert["scale_2"])
        / (1 + np.exp(-vert["opacity"]))
    )
    buffer = BytesIO()
    for idx in sorted_indices:
        v = plydata["vertex"][idx]
        pos.append(np.array([v["x"], v["y"], v["z"]], dtype=np.float32))
        scales.append(np.exp(
            np.array(
                [v["scale_0"], v["scale_1"], v["scale_2"]],
                dtype=np.float32,
            )
        ))
        rot.append(np.array(
            [v["rot_0"], v["rot_1"], v["rot_2"], v["rot_3"]],
            dtype=np.float32,
        ))
        SH_C0 = 0.28209479177387814
        color.append(np.array(
            [
                0.5 + SH_C0 * v["f_dc_0"],
                0.5 + SH_C0 * v["f_dc_1"],
                0.5 + SH_C0 * v["f_dc_2"],
                1 / (1 + np.exp(-v["opacity"])),
            ]
        ))
       
        

    return np.vstack(pos),np.vstack(scales),np.vstack(color),np.vstack(rot)



In [250]:
import math
def compute_cov_3d(scale, mod, rot):
    # Create scaling matrix
    S = np.eye(3) * mod * scale

    # Normalize quaternion to get a valid rotation
    q = rot / np.linalg.norm(rot)
    r, x, y, z = q

    # Compute rotation matrix from quaternion
    R = np.array([
        [1.0 - 2.0 * (y * y + z * z), 2.0 * (x * y - r * z), 2.0 * (x * z + r * y)],
        [2.0 * (x * y + r * z), 1.0 - 2.0 * (x * x + z * z), 2.0 * (y * z - r * x)],
        [2.0 * (x * z - r * y), 2.0 * (y * z + r * x), 1.0 - 2.0 * (x * x + y * y)]
    ])

    M = S @ R  # Matrix multiplication

    # Compute 3D world covariance matrix Sigma
    Sigma = M.T @ M
    cov_3d = np.zeros(6, dtype=float)
    # Covariance is symmetric; store upper right
    cov_3d[0] = Sigma[0, 0]
    cov_3d[1] = Sigma[0, 1]
    cov_3d[2] = Sigma[0, 2]
    cov_3d[3] = Sigma[1, 1]
    cov_3d[4] = Sigma[1, 2]
    cov_3d[5] = Sigma[2, 2]

    # Covariance is symmetric; store upper right
    return cov_3d

def compute_cov_2d(mean,fx,fy,tan_fovx,tan_fovy,cov_3d,viewmat):

    t = np.matmul(viewmat[0:3,0:3].T , mean).T

 
    limx = 1.3 * tan_fovx
    limy = 1.3 * tan_fovy
    txtz = t[0] / t[2]
    tytz = t[1] / t[2]
    t[0] = min(limx, max(-limx, txtz)) * t[2]
    t[1] = min(limy, max(-limy, tytz)) * t[2]

    # Compute Jacobian matrix J
    J = np.array([
        [fx / t[2], 0.0, - (fx * t[0]) / (t[2] ** 2)],
        [0.0, fy / t[2], - (fy * t[1]) / (t[2] ** 2)],
        [0, 0, 0]
    ])


    W = viewmat[0:3,0:3].T
    # Compute T matrix
    T = W @ J  # Matrix multiplication
    # Construct Vk matrix (3D covariance matrix)
    Vrk = np.array([
        [cov_3d[0], cov_3d[1], cov_3d[2]],
        [cov_3d[1], cov_3d[3], cov_3d[4]],
        [cov_3d[2], cov_3d[4], cov_3d[5]]
    ])

 

    # Compute the covariance matrix
    cov = T.T @ Vrk.T @ T

    # Apply low-pass filter: ensure Gaussian is at least one pixel wide/high
    # cov[0, 0] += 0.3
    # cov[1, 1] += 0.3

    return np.array([float(cov[0, 0]), float(cov[0, 1]), float(cov[1, 1])])



def focal2fov(focal, pixels):
    return 2*math.atan(pixels/(2*focal))

def calc_det(cov):
    return (cov[0] * cov[2] - cov[1] * cov[1])

def calc_conic(cov,det):
    det_inv = 1 / det
    return  [cov[2] * det_inv, -cov[1] * det_inv, cov[0] * det_inv ]

In [251]:

image_width = 160
image_height = 160

fx = frames['P1407CAM1.jpg'].K_crop[0,0]
fy = frames['P1407CAM1.jpg'].K_crop[1,1]

tan_fovx = focal2fov(fx, image_width)
tan_fovy = focal2fov(fy, image_height)
viewmat = frames['P1407CAM1.jpg'].world_to_cam



input_file = "D:/Documents/lr_gs/gray_dense/point_cloud_15.ply"
pos,scale,color,rot = process_ply_to_splat(input_file)

In [249]:
fx

-5308.420801138058

In [252]:

mod = 1.0

idx = -1
mean = pos

def get_cov(scale,rot,mean,viewmat,fx,fy,tan_fovx,tan_fovy):
    cov_3d = compute_cov_3d(scale, 1, rot)
    cov = compute_cov_2d(mean,fx,fy,tan_fovx,tan_fovy,cov_3d,viewmat)
    return cov

# det = calc_det(cov)
# conic = calc_conic(cov,det)

covs = np.vstack([get_cov(scale,rot,mean,viewmat,fx,fy,tan_fovx,tan_fovy) for scale,rot,mean in zip(scale,rot,mean)])

homo_pos = frames['P1407CAM2.jpg'].add_homo_coords(pos)
proj = frames['P1407CAM2.jpg'].project_on_image(homo_pos,croped_camera_matrix = True)


In [253]:
indices1 = (color[:,2] >= 0.1) & (color[:,2] < 0.98) & (color[:,3] < 1)

indices = (np.abs(covs[:,1])<100) & (np.abs(covs[:,2])<200) & (np.abs(covs[:,0])<200)
for_ell = covs[indices]
for_ell_proj = proj[indices]
color_ell = color[indices]

In [254]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

def plot_gaussian_ellipse(mean, cov_2d, ax=None, n_std=1.0, **kwargs):
    """Plot a 2D Gaussian as an ellipse."""
    if ax is None:
        ax = plt.gca()

    # Covariance matrix as 2x2
    cov_matrix = np.array([[cov_2d[0], cov_2d[1]], 
                           [cov_2d[1], cov_2d[2]]])

    # Eigenvalues and eigenvectors for the orientation and size
    eigenvals, eigenvecs = np.linalg.eigh(cov_matrix)
    order = eigenvals.argsort()[::-1]
    eigenvals, eigenvecs = eigenvals[order], eigenvecs[:, order]

    # Calculate the angle of the ellipse (in degrees)
    vx, vy = eigenvecs[:, 0]
    theta = np.degrees(np.arctan2(vy, vx))

    # Ellipse parameters
    width, height = 2 * n_std * np.sqrt(eigenvals)
    
    # Create the ellipse
    ell = Ellipse(xy=mean, width=width, height=height, angle=theta, **kwargs)
    
    ax.add_patch(ell)
    return width, height,theta


# Plot the Gaussian as an ellipse
fig, ax = plt.subplots()
ax.set_xlim(0, 160)
ax.set_ylim(0,160)
plt.scatter(proj[indices1,0],proj[indices1,1],s = 1)

# Plot the Gaussian with 1 standard deviation
b = [plot_gaussian_ellipse(for_ell_proj, for_ell, ax=ax, facecolor = color_ell[0:3], alpha=color_ell[3]) for for_ell_proj,for_ell,color_ell in zip(for_ell_proj[0:1000],for_ell[0:1000],color_ell[0:1000])]
# Show the plot
plt.grid(True)
plt.show()

In [165]:
b

[(27.011463124684077, 16.709646356910195, -137.8207498754393),
 (28.756192412815153, 12.99058287092817, 153.45190006528466),
 (22.044058028567527, 13.849853034627683, -151.09922207549772),
 (22.72898211212362, 14.69376482454708, 139.9447880976183),
 (27.01768953107769, 12.867134692431609, 57.17065320728768),
 (27.57411474426741, 14.822620126002125, 161.77745110229282),
 (20.966135105612178, 13.433454458132214, -149.11727047788307),
 (27.71185652199112, 12.198979078554158, 134.31121343755052),
 (27.454598123452634, 6.326239952791792, -157.06122691892938),
 (21.857813873792466, 10.531710762264144, -146.14617400019768),
 (32.65912570795273, 20.987069062514067, -135.5827150586649),
 (28.314057543326346, 21.664584144151373, 95.95797730991727),
 (24.716109372284425, 23.383596823169043, 56.288512366851386),
 (14.99321819805404, 12.095557562925173, 177.63779873200465),
 (28.660140750604302, 13.304705459125573, -150.8465293240579),
 (33.330053492739935, 7.2822776062290995, 132.2028431916446),
 

In [122]:
for_ell[:,0]*for_ell[:,2] - for_ell[:,1]*for_ell[:,1] 

array([[ 1.31638512e+02,  5.60281618e+01,  1.20569344e+02],
       [ 1.73860418e+02, -6.57871162e+01,  7.50580432e+01],
       [ 1.04310354e+02,  3.11110422e+01,  6.51293764e+01],
       ...,
       [ 5.11134414e-01, -1.07170627e-01,  3.54871447e-01],
       [ 4.21098920e+00,  9.85440507e+00,  2.51314231e+01],
       [ 1.18874675e+00,  2.02123437e+00,  4.89711273e+00]])

In [120]:
n_std = 1
cov_2d = for_ell
# Covariance matrix as 2x2
cov_matrix = np.array([[cov_2d[0], cov_2d[1]], 
                        [cov_2d[1], cov_2d[2]]])

# Eigenvalues and eigenvectors for the orientation and size
eigenvals, eigenvecs = np.linalg.eigh(cov_matrix)
order = eigenvals.argsort()[::-1]
eigenvals, eigenvecs = eigenvals[order], eigenvecs[:, order]

# Calculate the angle of the ellipse (in degrees)
vx, vy = eigenvecs[:, 0]
theta = np.degrees(np.arctan2(vy, vx))

# Ellipse parameters
width, height = 2 * n_std * np.sqrt(eigenvals)

array([[ 1.31638512e+02,  5.60281618e+01,  1.20569344e+02],
       [ 1.73860418e+02, -6.57871162e+01,  7.50580432e+01],
       [ 1.04310354e+02,  3.11110422e+01,  6.51293764e+01],
       ...,
       [ 5.11134414e-01, -1.07170627e-01,  3.54871447e-01],
       [ 4.21098920e+00,  9.85440507e+00,  2.51314231e+01],
       [ 1.18874675e+00,  2.02123437e+00,  4.89711273e+00]])

In [119]:
for_ell_proj

array([[ 84.19078169,  47.06180881],
       [ 76.25174261,  87.23355188],
       [ 84.18230826,  47.46902143],
       ...,
       [ 44.42871227,  63.13493924],
       [117.26919451,  99.38322307],
       [117.19633068,  99.37221166]])

In [221]:
indices

array([False, False, False, ..., False, False, False])

In [226]:
plydata = PlyData.read(input_file)
im_name = list(frames.keys())[0]

# projection
fig,axs = plt.subplots(2,2)
for cam in range(4):
    image = f'{im_name.split("CAM")[0]}CAM{cam+1}.jpg'
    indices =(color[:,0] < 1) & (color[:,1] < 1) &(color[:,2] < 1) & (color[:,3] < 1) & (color[:,0] > 0) & (color[:,1] > 0) &(color[:,2] > 0) 

    splat = pos[indices,:]
    colors = color[indices, :]  # Filtered colors (RGB or RGBA)

    homo_voxels_with_idx = frames[image].add_homo_coords(splat[:,0:3])
    proj = frames[image].project_on_image(homo_voxels_with_idx,croped_camera_matrix = True)
    axs[cam // 2,cam % 2].imshow(frames[image].croped_image)
    axs[cam // 2,cam % 2].scatter(proj[:,0],proj[:,1],s = 5,c = colors)